In [8]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
parasharmanas_movie_recommendation_system_path = kagglehub.dataset_download('parasharmanas/movie-recommendation-system')

print('Data source import complete.')


Data source import complete.


# Introduction

In this notebook, we will use the TF-IDF (term frequency-inverse document frequency) method to recommend movies based on how similar these movies are.

Users can give a title of movie. This system can search for the movies with similar title and make a recommendation based on the ratings and genres.

# 1. Data Loading

We import the necessary libraries:

In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

We look into the movies.csv first:

In [10]:
df1 = pd.read_csv("/content/sample_data/movies.csv")
print(df1.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [11]:
df1.isna().sum()

,0
movieId,0
title,0
genres,0


We only needs the words and numbers in the title, so we define a clean_title function to clean the title. The genres are separated by a "|". We replace them by space.

In [12]:
import re

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)

In [13]:
df1['genres_list'] = df1['genres'].str.replace('|', ' ')
df1['clean_title'] = df1['title'].apply(clean_title)

movies_data = df1[['movieId', 'clean_title', 'genres_list']]
print(movies_data.head())

   movieId                       clean_title  \
0        1                    Toy Story 1995   
1        2                      Jumanji 1995   
2        3             Grumpier Old Men 1995   
3        4            Waiting to Exhale 1995   
4        5  Father of the Bride Part II 1995   

                                   genres_list  
0  Adventure Animation Children Comedy Fantasy  
1                   Adventure Children Fantasy  
2                               Comedy Romance  
3                         Comedy Drama Romance  
4                                       Comedy  


Now we look into the ratings.csv:

In [15]:
df2 = pd.read_csv("/content/sample_data/ratings.csv")
print(df2.head())

   userId  movieId  rating     timestamp
0       1      296     5.0  1.147880e+09
1       1      306     3.5  1.147869e+09
2       1      307     5.0  1.147869e+09
3       1      665     5.0  1.147879e+09
4       1      899     3.5  1.147869e+09


In [16]:
df2.isna().sum()

,0
userId,0
movieId,0
rating,1
timestamp,1


We can safely remove the "timestamp" column.

In [17]:
ratings_data = df2.drop(['timestamp'], axis=1)
print(ratings_data.head())

   userId  movieId  rating
0       1      296     5.0
1       1      306     3.5
2       1      307     5.0
3       1      665     5.0
4       1      899     3.5


Finally, we merge two dataframes.

In [18]:
combined_data = ratings_data.merge(movies_data, on='movieId')
print(combined_data.head())

   userId  movieId  rating                                 clean_title  \
0       1      296     5.0                           Pulp Fiction 1994   
1       1      306     3.5  Three Colors Red Trois couleurs Rouge 1994   
2       1      307     5.0  Three Colors Blue Trois couleurs Bleu 1993   
3       1      665     5.0                            Underground 1995   
4       1      899     3.5                     Singin in the Rain 1952   

                   genres_list  
0  Comedy Crime Drama Thriller  
1                        Drama  
2                        Drama  
3             Comedy Drama War  
4       Comedy Musical Romance  


# 2. Definition of the Search function

We use the TF-IDF matrix to search for 5 movies with similar title.

In [19]:
vectorizer_title = TfidfVectorizer(ngram_range=(1,2))

tfidf_title = vectorizer_title.fit_transform(movies_data['clean_title'])

def search_by_title(title):
    title = clean_title(title)
    query_vec = vectorizer_title.transform([title])
    similarity = cosine_similarity(query_vec, tfidf_title).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies_data.iloc[indices][::-1]
    return results

movie_results = search_by_title("Toy Story")
print(movie_results)

       movieId               clean_title  \
14813    78499          Toy Story 3 2010   
3021      3114          Toy Story 2 1999   
0            1            Toy Story 1995   
59767   201588          Toy Story 4 2019   
20497   106022  Toy Story of Terror 2013   

                                            genres_list  
14813  Adventure Animation Children Comedy Fantasy IMAX  
3021        Adventure Animation Children Comedy Fantasy  
0           Adventure Animation Children Comedy Fantasy  
59767               Adventure Animation Children Comedy  
20497                         Animation Children Comedy  


Similarly, we can find movies with similar genres.

In [20]:
vectorizer_genres = TfidfVectorizer(ngram_range=(1,2))

tfidf_genres = vectorizer_genres.fit_transform(movies_data['genres_list'])

def search_similar_genres(genres):
    query_vec = vectorizer_genres.transform([genres])
    similarity = cosine_similarity(query_vec, tfidf_genres).flatten()
    indices = np.argpartition(similarity, -10)[-10:]
    results = movies_data.iloc[indices][::-1]
    return results

gen = 'Adventure Comedy'
print(search_similar_genres(gen))

       movieId                                     clean_title  \
36401   150138             The Four Charlots Musketeers 2 1973   
36434   150232                               Bad Hair Day 2015   
14673    77841  St Trinians 2 The Legend of Frittons Gold 2009   
931        952                Around the World in 80 Days 1956   
37581   152968                                     Lusers 2015   
37582   152970                  Hunt for the Wilderpeople 2016   
14004    72559              Last Remake of Beau Geste The 1977   
60085   202419                                Rum Runners 1971   
13828    71560  Abbott and Costello in the Foreign Legion 1950   
13760    71278                         Mr Robinson Crusoe 1932   

            genres_list  
36401  Adventure Comedy  
36434  Adventure Comedy  
14673  Adventure Comedy  
931    Adventure Comedy  
37581  Adventure Comedy  
37582  Adventure Comedy  
14004  Adventure Comedy  
60085  Adventure Comedy  
13828  Adventure Comedy  
13760  Advent

# 3. Make the recommendation

The following function calculates scores for recommendation.

In [21]:
def scores_calculator(movie_id):
    #find the recommendations from users who like the same movie
    similar_users = combined_data[(combined_data['movieId']== movie_id) & (combined_data['rating']>=4)]['userId'].unique()
    similar_user_recs = combined_data[(combined_data['userId'].isin(similar_users)) & (combined_data['rating']>=4)]['movieId']
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    #print(similar_user_recs)

    #find the recommendations from all users who have watch the movies above
    all_users = combined_data[(combined_data['movieId'].isin(similar_user_recs.index)) & (combined_data['rating']>=4)]
    all_users_recs = all_users['movieId'].value_counts() / all_users['userId'].nunique()
    #print(all_users_recs)

    genres_of_selected_movie = combined_data[combined_data['movieId']==movie_id]['genres_list'].unique()
    genres_of_selected_movie = np.array2string(genres_of_selected_movie)
    movies_with_similar_genres = search_similar_genres(genres_of_selected_movie)

    indices = []
    for index in movies_with_similar_genres[(movies_with_similar_genres['movieId'].isin(similar_user_recs.index))]['movieId']:
        indices.append(index)

    #times a factor 1.5 to movies with similar genres and similar users
    similar_user_recs.loc[indices] = similar_user_recs.loc[indices]*1.5

    #times a factor 0.9 to movies with similar genres and all users
    indices = []
    for index in movies_with_similar_genres[(movies_with_similar_genres['movieId'].isin(all_users_recs.index))]['movieId']:
        indices.append(index)
    all_users_recs.loc[indices] = all_users_recs.loc[indices]*0.9

    rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
    rec_percentages.columns = ['similar', 'all']
    rec_percentages['score'] = rec_percentages['similar'] / rec_percentages['all']

    rec_percentages = rec_percentages.sort_values('score', ascending=False)
    return rec_percentages

scores_calculator(3114)

,similar,all,score
movieId,,,
65577,0.013393,0.000757,17.689732
115875,0.003348,0.000189,17.689732
89069,0.002232,0.000210,10.613839
84989,0.002232,0.000210,10.613839
6236,0.002232,0.000210,10.613839
...,...,...,...
127202,0.002232,0.004206,0.530692
97188,0.002232,0.004206,0.530692
9010,0.002232,0.004416,0.505421


We list 10 movies with the highest score to user

In [22]:
def recommendation_results(user_input, title=0):
    # user_input = clean_title(user_input)
    title_candidates = search_by_title(user_input)
    movie_id = title_candidates.iloc[title]['movieId']
    scores = scores_calculator(movie_id)
    results = scores.head(10).merge(movies_data, left_index=True, right_on='movieId')[['clean_title', 'score', 'genres_list']]
    resutls = results.rename(columns={'clean_title': 'title', 'genres_list': 'genres'}, inplace=True)
    return results

user_input = "Jalsa"
print("Are you looking for (please choose a number): ")
for i in range(5):
    print(i, ": ", search_by_title(user_input)['clean_title'].iloc[i])

title = 0
if int(title) in range(5):
    print("We have following recommendations: ")
    print(recommendation_results(user_input, int(title)))
else:
    print("Sorry! please try again!")
# print("================================================")
# print("Recommendation_results: ")
# print(recommendation_results(user_input))

Are you looking for (please choose a number): 
0 :  Jalsa 2008
1 :  I Will Fight No More Forever 1975
2 :  Carmen 1918
3 :  High School 2010
4 :  Gorko 2013
We have following recommendations: 
Empty DataFrame
Columns: [title, score, genres]
Index: []
